# Notebook setup

## Import basic libraries

In [1]:
import os
import sys
import numpy as np
import json
import subprocess
import operator
from operator import itemgetter
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from setup_geometry_interactions_db import \
    Json,Tags,RigidBody,RadialProfiles,Miscellaneous,Interhelix_Interactions, Base

> _NOTE_ Make sure that you install `requirements.txt`

## Initialise database session

In [ ]:
folder = 'data'
dbfile = 'mutants_docked_geometry-interactions.db' # database
path   = os.path.join(folder, dbfile)

In [2]:
engine = create_engine('sqlite:///'+path)
Base.metadata.bind = engine
DBSession = sessionmaker()
DBSession.bind = engine
session = DBSession()

# Pore geometry analysis

Base function

In [3]:
def compute_radial_metrics(profile_Y, metric):
    """
    Function to compute several metrics given a radial profile of a symmetric docking model.
    
    Parameters
    ----------
    profile_Y : ndarray, shape(n,)
        radial profile.
        
    metric: string
        metric name to be measured from given radial profile.
    
    Returns
    -------
        value (float) of selected metric
    """
    if metric == 'length':
        return max(profile_Y) - min(profile_Y)
    
    elif metric == 'mean':
        return numpy.mean(profile_Y)
    
    elif metric == 'std':
        return numpy.std(profile_Y)
    
    elif metric == 'cv':
        return numpy.std(profile_Y)/numpy.mean(profile_Y)
    
    elif metric == 'min':
        return min(profile_Y)
    
    elif metric == 'max':
        return max(profile_Y)

Measure all average _Euler angles_ and _profile metrics_ for all mutant models, from their primitive and vdW radial profiles.

In [4]:
mutant_names = ['cWza','cWza-K375C','cWza-S355C','cWza-Y373C']

In [5]:
S_mutants = {} # dict for data storage
metrics = ['length','mean','std','cv','min','max']

for i in range(len(mutant_names)):
    mutant = mutant_names[i]
    S_mutants[mutant] = {} 
    group = 'None' # if model tagged
    Theta = []; Psi = []; Phi = [] # Euler angles lists
    
    model_ids = [x[0] for x in session.query(Tags.id).filter_by(mutant = mutant)]
    
    # Create lists for radial metrics
    for j in range(len(model_ids)):
        id = model_ids[j]
        
        # Euler angles: mean values for all chains
        euler_angles_all = session.query(RigidBody.euler_angles).filter_by(id = id).all()[0][0]
        euler_angles = {
            'tilt' : numpy.mean(euler_angles_all[0]),
            'interchain' : numpy.mean(euler_angles_all[1]),
            'facing' : numpy.mean(euler_angles_all[2])
            }
        
        # Primitive profiles: radial and length metrics
        primitive_profile = session.query(RadialProfiles.primitive).filter_by(id = id).all()[0][0]
        X, Y = primitive_profile
        primitive_R_metrics = {}
        
        for key in metrics:
            if key != 'length':
                S = Y # radial coordinates
            else:
                S = X # longitudinal coords
            primitive_R_metrics[key] = compute_radial_metrics(S, key)
        
        # VdW profiles: radial and length metrics
        vdw_profile = session.query(RadialProfiles.vdw_aa_lower).filter_by(id = id).all()[0][0]
        X,Y = vdw_profile
        vdw_R_metrics = {} 
        
        for key in metrics:
            if key != 'length':
                S = Y
            else:
                S = X
            vdw_R_metrics[key] = compute_radial_metrics(S,key)
            
        # Overall data as single list
        model_data = {
            'euler angles': euler_angles,
            'primitive metrics': primitive_R_metrics,
            'vdw metrics': vdw_R_metrics
        }
        
        # Assign data dict according to model ID
        S_mutants[mutant][id] = model_data
        

/home/ba13026/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SAWarning: TypeDecorator Json() will not produce a cache key because the ``cache_ok`` flag is not set to True.  Set this flag to True if this type object's state is safe to use in a cache key, or False to disable this warning.
/home/ba13026/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SAWarning: TypeDecorator Json() will not produce a cache key because the ``cache_ok`` flag is not set to True.  Set this flag to True if this type object's state is safe to use in a cache key, or False to disable this warning.
/home/ba13026/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: SAWarning: TypeDecorator Json() will not produce a cache key because the ``cache_ok`` flag is not set to True.  Set this flag to True if this type object's state is safe to use in a cache key, or False to disable this warning.


Save data as JSON file. This file should accompany this Notebook by default. 

```python
folder = 'data'
filename = 'mutants_docked_geometry-interactions_post-processed.json'
path   = os.path.join(folder, filename)

with open(path, 'w') as fp:
    json.dump(S_mutants, fp)
```

Load data from JSON file using 

```python
folder = 'data'
filename = 'mutants_docked_geometry-interactions_post-processed.json'
path   = os.path.join(folder, filename)

with open(file, 'r') as fp:
    data = json.load(fp)
```